In [ ]:
# Pour rouler dans Google Colab, executez cette cellule en premier

!git clone https://github.com/Algolab-Sherhack-2024/defi-principal.git
import sys
sys.path.insert(0,'/content/defi-principal')
!pip install -r defi-principal/requirements.txt

# Problème 1 : Les flottes d'autobus

# Import des librairies nécessaires

La cellule qui suit contient les imports nécessaire pour l'execution de ce notebook. Vous êtes invité(e)s à modifier ce notebook comme il vous chante.

In [16]:
# Affichage du graphe de problème
import matplotlib.pyplot as plt
import networkx as nx

# Calcul matriciel et optimisation classique
import numpy as np
from scipy.optimize import minimize

# Génération de circuit quantique
from qiskit import QuantumCircuit

# Structure pour l'écriture d'hamiltoniens
from qiskit.quantum_info import SparsePauliOp

# Méthode de génération du circuit de QAOA à optimiser
from qiskit.circuit.library import QAOAAnsatz

# Outils pour l'execution de circuits quantiques
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_ibm_runtime import SamplerV2 as Sampler

# Transpilation de circuits quantiques pour les machines et simulateurs d'IBM
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Simulateur de processeur quantiques
from qiskit_aer import AerSimulator

# Simulateurs de processeurs quantiques existants
from qiskit_ibm_runtime.fake_provider import FakeNairobiV2  # , FakeQuebec

# Visualisation de distributions de probabilités
from qiskit.visualization import plot_histogram

# Introduction

La société de transport de Sherbrooke (STS) fait face à un problème d'optimisation pour l'organisation des autobus qui desservent la ville. Nous vous proposons un cas simplifié pour expérimenter la résolution de problèmes d'optimisation combinatoire en utilisant l'algorithme quantique **QAOA**.

## Énoncé du problème

>La STS souhaite ajouter cinq autobus de cinq lignes différentes afin de répondre à la demande croissante des usagers, pour les départs de 08:00 et 08:30. Étant donné que certains trajets se chevauchent, l'objectif est de répartir ces autobus en deux groupes, de sorte que les bus partageant des arrêts communs aient, dans la mesure du possible, des heures de départ différentes. 

Pour modéliser ce problème, nous représentons chaque bus par un nœud dans un graphe. Une arête entre deux nœuds indique un chevauchement entre les lignes des autobus correspondants.

<p align="center">
<img src="https://github.com/Algolab-Sherhack-2024/defi-principal/blob/main/probleme_img_1.jpg?raw=true" alt="probleme 1" width="250"/>
</p>

## Affichage du graphe avec Python

La librairie Networkx est utilisée pour visualiser le graphe correspondant à ce problème. Le code fourni peut également être utilisé pour d'autres problèmes du hackathon. L'option d'ajout et d'affichage de poids sur les arêtes est disponible, mais elle est désactivée par défaut dans le code.

La visualisation du graphe de problème est optionnelle, mais peut potentiellement vous aider dans la résolution des prochains défis. 

## Objectif

Votre mission est d'optimiser les hyperparamètres du circuit de QAOA afin de résoudre ce problème de manière efficace. Le pipeline de la solution avec QAOA est déjà fourni, mais il vous est demandé de trouver les meilleurs paramètres de circuit, ainsi que le nombre optimal de couches du circuit.

### Livrables attendus :
- Une série de paramètres optimaux pour le circuit QAOA.
- Le nombre de couches du circuit QAOA.

#### Format de soumission des livrables :
Un fichier de format `.npz` est attendu. Vous pouvez générer ce fichier avec les informations requises en utilisation la méthode `sauvegarder_res` disponible dans `utils.py`.

### Évaluation
Une partie de la correction se fera en exécutant votre circuit QAOA avec le nombre de couches et les paramètres fournis. Un score sera calculé à partir de la solution obtenue à l'aide de la méthode de notation prédéfinie `calc_score` disponible dans `utils.py`.

Les juges évalueront également la qualité du code et l'originalité de votre optimisation de paramètres pour QAOA. 

In [ ]:
# Instancier un graphe avec Networkx
G = nx.Graph()
# Ajouter les arêtes du graphe et leur poids, au besoin. Une liste de la forme [(noeud_A, noeud_B, poids_arête_AB), ...].
G.add_weighted_edges_from([(0, 1, 1), (1, 2, 1), (2, 3, 1), (3, 0, 1), (0, 4, 1), (4, 1, 1)])
# Définir les positions des noeuds pour le dessin du graphe
pos = {0: (-1, 0), 1: (1, 0), 2: (1, -1), 3: (-1, -1), 4: (0, 1)}
# Définir les options de dessin
options = {"node_size": 1000, "node_color": "white", "edgecolors": "black", "linewidths": 2, "width": 2}
# Dessiner le graphe
nx.draw_networkx(G, pos, **options)
# Dessiner, au besoin, le poids de chaque arête
# nx.draw_networkx_edge_labels(G, pos, nx.get_edge_attributes(G, "weight"))

# Afficher le dessin du graphe
ax = plt.gca()
ax.margins(0.20)
plt.axis("off")
plt.show()

# Fonction de coût

Avant de résoudre le problème à l'aide de QAOA, nous devons d'abord définir une fonction de coût qui représente notre objectif d'optimisation.

## Qubits et valeurs moyennes

Le problème de répartition des autobus doit être reformulé en termes de qubits et de portes logiques quantiques pour être résolu par un algorithme quantique tel que **QAOA**. Chaque nœud du graphe représente un autobus, et nous utilisons un qubit pour encoder chaque nœud. Ainsi, le nombre de qubits dans notre circuit correspond au nombre de nœuds (ou d’autobus) dans le graphe.

Pour capturer les relations entre les noeuds, nous définissons les interactions entre les qubits en fonction des arêtes du graphe. Par exemple, une chaîne de Pauli telle que $IIIZZ$ encode une arête entre les qubits (nœuds) 0 et 1, où chaque matrice de Pauli $Z_i$ correspond à une mesure sur un qubit $i$.

Lorsque les deux qubits 0 et 1 sont dans le même groupe, l'observable $IIIZZ$ associée à cette arête prendra la valeur $1$. Si les qubits sont dans des groupes différents, l'observable prendra la valeur $-1$. Cette formulation nous permet d’évaluer si deux nœuds (autobus) sont correctement séparés.

Au besoin, référez-vous à la présentation pour des rappels sur la mesure d'observables. 

## Expression de l’Hamiltonien

L’Hamiltonien, qui représente la fonction de coût du problème, est exprimé comme la somme des chaines de Pauli associées à chaque paire de qubits connectés par une arête dans le graphe. Pour deux qubits $i$ et $j$ connectés, l'interaction est modélisée par le terme $Z_i Z_j$.

L'Hamiltonien pour l’ensemble du graphe est donc :

$$
H = \sum_{(i,j) \in E} Z_i Z_j
$$

où $E$ est l'ensemble des arêtes du graphe. Cette somme couvre toutes les paires de nœuds $i$ et $j$ qui partagent une arête, c’est-à-dire tous les autobus ayant des trajets qui se chevauchent.

> L’objectif est de minimiser cette fonction pour séparer les autobus en deux groupes de manière optimale.

**Note :** Cela correspond à un problème de Max-Cut simple. 

Nous utilisons la structure `SparsePauliOp` pour instancier l'Hamiltonien. 

In [ ]:
# Instancier un SparsePauliOp comme une somme de chaines de Pauli multipliées par un certain coefficient (ici 1).
H = SparsePauliOp.from_list(
    [("IIIZZ", 1.0), ("IIZZI", 1.0), ("IZZII", 1.0), ("IZIIZ", 1.0), ("ZIIIZ", 1.0), ("ZIIZI", 1.0)]
)
# Afficher le SparsePauliOp
print(H)

# Vérification classique

Le problème peut être représenté sous forme matricielle. Étant donné sa petite taille, il est possible de diagonaliser cette matrice pour obtenir une solution exacte. Vous pouvez utiliser cette méthode de diagonalisation pour vérifier la validité de votre solution quantique.

**Note** : Cette méthode se retrouve dans `utils.py` et peut-être réutilisée pour vérifier vos solutions aux prochains problèmes également. 

In [19]:
def calc_solutions_exactes(hamiltonien: SparsePauliOp) -> tuple[float, list[str]]:
    """Calcul classique des solutions de l'hamiltonien fourni en entrée.
    Cela est fait en diagonalisant la matrice de l'hamiltonien.

    Args:
        hamiltonien (SparsePauliOp): Hamiltonien à diagonaliser exprimé sous la forme d'une somme de chaines de Pauli.

    Returns:
        tuple[float, list[str]]:
            - Coût minimal obtenue (float)
            - Liste de solutions binaires associées au coût minimal
    """
    # Transformer l'hamiltonien en matrice
    mat_hamiltonien = np.array(hamiltonien.to_matrix())
    # Diagonaliser la matrice pour en extraire vecteurs et valeurs propres
    valeurs_propres, vecteurs_propres = np.linalg.eig(mat_hamiltonien)

    # Index des valeurs propres minimales
    min_val_propre = np.where(valeurs_propres == np.min(valeurs_propres))[0]
    # Solutions minimales associées aux valeurs propres minimales
    sol_binaires = [bin(idx).lstrip("-0b").zfill(hamiltonien.num_qubits) for idx in min_val_propre]

    # Coût and chaines binaires de toutes les meilleures solutions
    return valeurs_propres[min_val_propre][0].real, sol_binaires

In [ ]:
# Vérifier que l'hamiltonien créé nous donne la solution attendue
cout_minimal, sol_binaires = calc_solutions_exactes(hamiltonien=H)

# Afficher le coût minimal obtenu
print("Coût minimal : ", cout_minimal)
# Afficher les solutions binaires (à lire de droite à gauche, du qubit 0 au qubit 4)
for i, sol in enumerate(sol_binaires):
    print("Solution à coût minimal {idx} : {solution}".format(idx=i, solution=sol))

# Cadre de solution avec QAOA

QAOA (Quantum Approximate Optimization Algorithm) est un algorithme variationnel quantique, utilisé ici pour résoudre notre problème d'optimisation combinatoire. Pour les détails théoriques de l'algorithme, référez-vous à la présentation de ce matin.

Dans cette première partie du défi, vous devez ajuster les hyperparamètres de votre exécution hybride pour résoudre efficacement le problème de gestion de flottes d’autobus. Les hyperparamètres à régler sont les suivants :

- Le nombre de couches dans le circuit QAOA.
- Les paramètres (angles) à insérer dans le circuit QAOA.
- Les options de l'optimiseur classique, utilisé pour ajuster les paramètres (angles) du QAOA.

Votre objectif est de trouver la meilleure combinaison de couches et de paramètres pour obtenir systématiquement des solutions au problème de MaxCut, lors de plusieurs exécutions du circuit QAOA.

## Circuit de QAOA

Dans les cellules suivantes, vous trouverez une structure proposée pour construire le circuit QAOA basé sur l'Hamiltonien du problème. Expérimentez avec le nombre de couches pour améliorer l'approximation de la solution.

À noter : Un nombre de couches trop faible peut ne pas fournir une solution stable, tandis qu'un nombre trop élevé augmente considérablement le temps d'exécution. L’idéal est de trouver un équilibre qui permet d’amplifier efficacement les solutions sans complexifier inutilement la simulation, afin de rester dans des délais raisonnables pour une exécution sur un ordinateur portable.

In [ ]:
# Identifier un nombre de couches (répétitions) pour QAOA
nb_couches = 1  # Modifier comme il vous plait

# Instancier le circuit de QAOA avec l'outil de Qiskit
circuit_qaoa = QAOAAnsatz(H, reps=nb_couches)

# Afficher, si désiré, le circuit de QAOA
circuit_qaoa.decompose(reps=1).draw(output="mpl", style="iqp")

In [22]:
# Définir un simulateur pour faire nos calculs de valeurs moyennes
backend = AerSimulator()  # À changer si désiré pour d'autres simulateurs / vrai ordinateur
estimateur = Estimator(mode=backend)

# Définir un outil de transpilation tel qu'exigé par Qiskit
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)

In [23]:
# Définir la fonction de coût classique à optimiser
def fonction_cout(
    params: list[complex], estimator: Estimator, circuit: QuantumCircuit, hamiltonien: SparsePauliOp
) -> float:
    """Fonction de cout qui calcule la valeur moyenne d'un observable (hamiltonien) pour un état donnée (circuit).
    Cette valeur moyenne représente le coût de la fonction de coût décrite par l'hamiltonien en entrée. Aussi, le
    circuit est paramétré et les paramètres sont définis dans le vecteurs params.
    Le tout est évalué à l'aide de l'estimateur (estimator).

    Args:
        params (list[complex]): Liste de paramètres à insérer dans le circuit en entrée
        estimator (Estimator): Calculateur utilisé pour estimer la valeur moyenne désirée.
        circuit (QuantumCircuit): Circuit paramétré de QAOA.
        hamiltonien (SparsePauliOp): Observable décrivant la fonction de coût du problème donné.

    Returns:
        float: Coût associé aux paramètres passés en entrée.
    """
    isa_psi = pm.run(circuit)
    isa_observables = hamiltonien.apply_layout(isa_psi.layout)

    job = estimator.run([(isa_psi, isa_observables, params)])
    cout = job.result()[0].data.evs
    return cout

## Paramètres du circuit de QAOA

La cellule suivante permet d'effectuer l'optimisation classique des paramètres (angles) du circuit QAOA. Ces paramètres sont initialisés avec un vecteur de zéros, mais il peut être *(très)* pertinent d'explorer d'autres initialisations pour améliorer les résultats de l'optimisation.

### Rappel de soumission :

Vous devez fournir une série de paramètres qui, lorsqu'insérés dans le circuit QAOA, permettent d'amplifier de manière significative les solutions du problème de MaxCut dans la distribution de probabilités des résultats.

### Optimiseur classique

L'optimiseur **Cobyla** est proposé pour ajuster les paramètres à l'aide de la fonction de coût définie précédemment. Vous êtes libre de modifier les options de Cobyla ou d'utiliser un autre optimiseur si vous le jugez plus approprié.

Certains optimiseurs sans calcul de gradient sont implémentés directement dans `Qiskit`.

In [ ]:
# Initialisation des paramètres du circuit de QAOA à utiliser en premier lieu
params_init = np.zeros(circuit_qaoa.num_parameters)

# Optimisation classique des paramètres du circuit de QAOA à l'aide de Scipy
res_opt = minimize(
    fonction_cout, params_init, args=(estimateur, circuit_qaoa, H), method="COBYLA"
)  # , options={"tol": 1e-14}


# Extraction des informations suite à l'optimisation
cout_opt = res_opt.fun  # Cout optimal trouvé
params_opt = res_opt.x  # Paramètres optimaux trouvés

# Affichage des résultats obtenus
print("Cout optimal trouvé :", cout_opt)
print("Paramètres optimaux trouvés :", params_opt)

# Évaluation de la solution 

Après avoir déterminé un nombre de couches et un vecteur de paramètres, un **sampler** est utilisé pour générer la distribution de probabilités associée au circuit QAOA ainsi configuré. L'objectif est de vérifier que les bonnes solutions à notre problème sont significativement amplifiées dans cette distribution.

C'est l'une des façons dont le score de votre optimisation des hyperparamètres sera déterminé.

In [ ]:
# Instancier un Sampler pour obtenir une distribution de probabilités
sampler = Sampler(mode=backend)
# Calcul de la distribution de probabilités avec les paramètres optimaux trouvés
circuit_qaoa_copie = circuit_qaoa.decompose(reps=2).copy()
circuit_qaoa_copie.measure_all()
circuit_qaoa_copie.draw()
counts = sampler.run([(circuit_qaoa_copie, params_opt)]).result()[0].data.meas.get_counts()
# Afficher la distribution de probabilités obtenue
plot_histogram(counts)

# Sauvegarde de vos résultats 

Finalement, n'oubliez pas de sauvegarder vos résultats à l'aide de la méthode `sauvegarder_res` disponible dans `utils.py`.

Donnez un **nom significatif** à votre fichier, par exemple `equipe_A_probleme_1.npz`. Vous pouvez vérifier le contenu de votre fichier `.npz` à l'aide de la méthode `lire_res` disponible dans `utils.py`.

In [ ]:
# Note. Pour rouler sur un vrai ordinateur, il vous faut changer de backend. Commencez par vous authentifier.
# Cette cellule n'est à roulà
# er qu'une seule fois.
!pip install qiskit_ibm_provider
from qiskit_ibm_provider import IBMProvider

# Enregistrez votre clé d'authentification une fois.
# IBMProvider.save_account(token="<IBM Quantum API key>")  # Mettre en commentaire cette cellule lorsque terminé.

In [27]:
service = QiskitRuntimeService(instance="ibm-q/open/main")
backend = service.backend("ibm_sherbrooke")  # Une façon de récupérer un backend
backend = service.least_busy(operational=True, min_num_qubits=5)  # Une autre façon de récupérer un backend


# Le reste (Sampler, Estimator) reste inchangé.